<a href="https://colab.research.google.com/github/rubiemaher/FinalProject331/blob/main/FinalProjectCOMP331.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#all imports here!!!

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import torch


In [ ]:
pip install transformers datasets sklearn torch

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:

#load dataset
dataset = load_dataset("christinacdl/clickbait_detection_dataset")



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.json: 0.00B [00:00, ?B/s]

val.json: 0.00B [00:00, ?B/s]

test.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/30296 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3787 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3787 [00:00<?, ? examples/s]

# DistilBERT

In [ ]:
#pretrained model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


#tokenization function
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=64)

#tokenize dataset
encoded_dataset = dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/30296 [00:00<?, ? examples/s]

Map:   0%|          | 0/3787 [00:00<?, ? examples/s]

Map:   0%|          | 0/3787 [00:00<?, ? examples/s]

In [ ]:
#prepare model w/ classification head
model = AutoModelForSequenceClassification.from_pretrained(

    model_name,
    num_labels=2,
    id2label={0: "NOT", 1: "CLICKBAIT"},
    label2id={"NOT": 0, "CLICKBAIT": 1},

    )
model.to("cuda")

#define metrics (to visualize the data later)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    prec = precision_score(labels, predictions)
    rec = recall_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#training args
training_args = TrainingArguments(
    output_dir="./clickbait_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True,
    dataloader_num_workers=2,
)

#trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    compute_metrics=compute_metrics,
)



In [ ]:
#train!!!
trainer.train()



/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rubypeeler (rubypeeler-occidental-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.090000,0.088040,0.971745,0.967901,0.979021,0.973429
2,0.034900,0.109531,0.975970,0.981360,0.973027,0.977176
3,0.003500,0.146642,0.974914,0.970400,0.982517,0.976421


TrainOutput(global_step=5682, training_loss=0.05826798029472105, metrics={'train_runtime': 331.7802, 'train_samples_per_second': 273.94, 'train_steps_per_second': 17.126, 'total_flos': 1504962116130816.0, 'train_loss': 0.05826798029472105, 'epoch': 3.0})

In [ ]:
#eval
results = trainer.evaluate(encoded_dataset["test"])
print(results)



{'eval_loss': 0.11217521131038666, 'eval_accuracy': 0.9743860575653551, 'eval_precision': 0.9793658782083543, 'eval_recall': 0.972027972027972, 'eval_f1': 0.97568312860366, 'eval_runtime': 3.4379, 'eval_samples_per_second': 1101.546, 'eval_steps_per_second': 68.937, 'epoch': 3.0}


In [ ]:
#use the model for inference!
def classify_title(title):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        inputs = tokenizer(title, return_tensors="pt", truncation=True, padding=True, max_length=64)
        inputs = {k: v.to(device) for k, v in inputs.items()}  # MOVE INPUTS TO GPU
        model.to(device)  # Make sure model is also on GPU

        outputs = model(**inputs)
        logits = outputs.logits.detach().cpu().numpy()
        pred = np.argmax(logits, axis=-1)[0]
        return model.config.id2label[pred]

In [ ]:
print(classify_title("You won't believe what happened next!"))

CLICKBAIT


# BERT-base

In [ ]:
model_name1 = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name1)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
model1 = AutoModelForSequenceClassification.from_pretrained(
    model_name1,
    num_labels=2,

    id2label={0: "NOT", 1: "CLICKBAIT"},
    label2id={"NOT": 0, "CLICKBAIT": 1},

    )
model1.to("cuda")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
trainer1 = Trainer(
    model=model1,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer1.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.095700,0.108418,0.972802,0.978338,0.970030,0.974166
2,0.058600,0.105636,0.974386,0.972236,0.979520,0.975865
3,0.005300,0.145469,0.976234,0.974206,0.981019,0.977601


TrainOutput(global_step=5682, training_loss=0.06265615012833704, metrics={'train_runtime': 587.3801, 'train_samples_per_second': 154.735, 'train_steps_per_second': 9.673, 'total_flos': 2989204699944960.0, 'train_loss': 0.06265615012833704, 'epoch': 3.0})

In [ ]:
results1 = trainer1.evaluate(encoded_dataset["test"])
print(results1)

{'eval_loss': 0.14515969157218933, 'eval_accuracy': 0.9754423026142065, 'eval_precision': 0.971358024691358, 'eval_recall': 0.9825174825174825, 'eval_f1': 0.9769058852743978, 'eval_runtime': 4.6306, 'eval_samples_per_second': 817.826, 'eval_steps_per_second': 51.182, 'epoch': 3.0}


# RoBERTa-base

In [ ]:
model_name2 = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name1)

In [ ]:
model2 = AutoModelForSequenceClassification.from_pretrained(
    model_name1,
    num_labels=2,
    id2label={0: "NOT", 1: "CLICKBAIT"},
    label2id={"NOT": 0, "CLICKBAIT": 1},

    )
model2.to("cuda")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
trainer2 = Trainer(
    model=model2,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer2.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.096800,0.095459,0.974650,0.978414,0.973526,0.975964
2,0.037100,0.097358,0.976234,0.974677,0.980519,0.977590
3,0.005800,0.141550,0.975970,0.973254,0.981518,0.977369


TrainOutput(global_step=5682, training_loss=0.06076325801624374, metrics={'train_runtime': 544.043, 'train_samples_per_second': 167.06, 'train_steps_per_second': 10.444, 'total_flos': 2989204699944960.0, 'train_loss': 0.06076325801624374, 'epoch': 3.0})

In [ ]:
results2 = trainer2.evaluate(encoded_dataset["test"])
print(results2)

{'eval_loss': 0.10492078959941864, 'eval_accuracy': 0.9754423026142065, 'eval_precision': 0.9755854509217738, 'eval_recall': 0.978021978021978, 'eval_f1': 0.9768021950611125, 'eval_runtime': 5.0388, 'eval_samples_per_second': 751.571, 'eval_steps_per_second': 47.035, 'epoch': 3.0}
